In [4]:
import sqlite3
import pandas as pd
import jdcal
from datetime import datetime

In [5]:
#get table name
conn = sqlite3.connect("../Data/FPA_FOD_20170508.sqlite")
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
cursor = conn.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('spatial_ref_sys',), ('spatialite_history',), ('sqlite_sequence',), ('geometry_columns',), ('spatial_ref_sys_aux',), ('views_geometry_columns',), ('virts_geometry_columns',), ('geometry_columns_statistics',), ('views_geometry_columns_statistics',), ('virts_geometry_columns_statistics',), ('geometry_columns_field_infos',), ('views_geometry_columns_field_infos',), ('virts_geometry_columns_field_infos',), ('geometry_columns_time',), ('geometry_columns_auth',), ('views_geometry_columns_auth',), ('virts_geometry_columns_auth',), ('sql_statements_log',), ('SpatialIndex',), ('ElementaryGeometries',), ('KNN',), ('Fires',), ('idx_Fires_Shape',), ('idx_Fires_Shape_node',), ('idx_Fires_Shape_rowid',), ('idx_Fires_Shape_parent',), ('NWCG_UnitIDActive_20170109',)]


In [6]:
#get table fire
conn = sqlite3.connect("../Data/FPA_FOD_20170508.sqlite")
df = pd.read_sql_query("SELECT FIRE_YEAR,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,DISCOVERY_DATE,FIRE_SIZE FROM 'Fires'",conn)

In [7]:
def julian_to_gregorian(julian_date):
    year, month, day, frac = jdcal.jd2gcal(jdcal.MJD_0, julian_date - jdcal.MJD_0)
    return datetime(year, month, day)
df['DISCOVERY_DATE'] = df['DISCOVERY_DATE'].apply(julian_to_gregorian)
df['year'] = df['DISCOVERY_DATE'].dt.year
df['month'] = df['DISCOVERY_DATE'].dt.month

In [8]:
state_to_id = {
  "AL": 1,
  "AZ": 2,
  "AR": 3,
  "CA": 4,
  "CO": 5,
  "CT": 6,
  "DE": 7,
  "FL": 8,
  "GA": 9,
  "ID": 10,
  "IL": 11,
  "IN": 12,
  "IA": 13,
  "KS": 14,
  "KY": 15,
  "LA": 16,
  "ME": 17,
  "MD": 18,
  "MA": 19,
  "MI": 20,
  "MN": 21,
  "MS": 22,
  "MO": 23,
  "MT": 24,
  "NE": 25,
  "NV": 26,
  "NH": 27,
  "NJ": 28,
  "NM": 29,
  "NY": 30,
  "NC": 31,
  "ND": 32,
  "OH": 33,
  "OK": 34,
  "OR": 35,
  "PA": 36,
  "RI": 37,
  "SC": 38,
  "SD": 39,
  "TN": 40,
  "TX": 41,
  "UT": 42,
  "VT": 43,
  "VA": 44,
  "WA": 45,
  "WV": 46,
  "WI": 47,
  "WY": 48,
  "AK": 50
}
df['state'] = df['STATE'].map(state_to_id)
df

,FIRE_YEAR,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,DISCOVERY_DATE,FIRE_SIZE,year,month,state
0,2005,Miscellaneous,40.036944,-121.005833,CA,2005-02-02,0.10,2005,2,4.0
1,2004,Lightning,38.933056,-120.404444,CA,2004-05-12,0.25,2004,5,4.0
2,2004,Debris Burning,38.984167,-120.735556,CA,2004-05-31,0.10,2004,5,4.0
3,2004,Lightning,38.559167,-119.913333,CA,2004-06-28,0.10,2004,6,4.0
4,2004,Lightning,38.559167,-119.933056,CA,2004-06-28,0.10,2004,6,4.0
...,...,...,...,...,...,...,...,...,...,...
1880460,2015,Missing/Undefined,40.481637,-122.389375,CA,2015-09-26,0.01,2015,9,4.0
1880461,2015,Miscellaneous,37.617619,-120.938570,CA,2015-10-05,0.20,2015,10,4.0
1880462,2015,Missing/Undefined,37.617619,-120.938570,CA,2015-05-02,0.10,2015,5,4.0
1880463,2015,Missing/Undefined,37.672235,-120.898356,CA,2015-10-14,2.00,2015,10,4.0


In [9]:
fire_summary = df.groupby(['state', 'year', 'month']).size().reset_index(name='count')
fire_summary_dict = {(row['state'], row['year'], row['month']): row['count'] for _, row in fire_summary.iterrows()}

In [10]:
fire_summary.isnull().values.any()

False

In [11]:
fire_summary

,state,year,month,count
0,1.0,1992,1,9
1,1.0,1992,2,27
2,1.0,1992,3,14
3,1.0,1992,4,18
4,1.0,1992,5,10
...,...,...,...,...
12080,50.0,2015,7,116
12081,50.0,2015,8,31
12082,50.0,2015,9,9
12083,50.0,2015,10,2


In [12]:
file_paths = ['climdiv-pcpnst-v1.0.0-20231106.txt', 'climdiv-pdsist-v1.0.0-20231106.txt',
              'climdiv-phdist-v1.0.0-20231106.txt', 'climdiv-sp01st-v1.0.0-20231106.txt',
              'climdiv-sp02st-v1.0.0-20231106.txt', 'climdiv-sp06st-v1.0.0-20231106.txt',
              'climdiv-sp12st-v1.0.0-20231106.txt', 'climdiv-sp24st-v1.0.0-20231106.txt',
              'climdiv-tmaxst-v1.0.0-20231106.txt', 'climdiv-tminst-v1.0.0-20231106.txt',
              'climdiv-tmpcst-v1.0.0-20231106.txt', 'climdiv-zndxst-v1.0.0-20231106.txt']

In [13]:
dic = {}
count = 0
for file_path in file_paths:
    with open('../Data/'+file_path, 'r') as file:
        for line in file:
            words = line.split()
            state = int(words[0][:3])
            year = int(words[0][-4:])
            if year <1992 or year>2015:
                continue
            if state>47:
                continue
            for month in range(1,13):
                temp = words[month]
                key = (state, year, month)
                dic[key] = dic.get(key,[]) + [temp]
    count+=1

In [14]:
df2 = pd.DataFrame.from_dict(dic,orient='index')
df2.columns = ["pcpnst","pdsist","phdist","sp01st","sp02st","sp06st","sp12st","sp24st","tmaxst","tminst","tmpcst","zndxst"]
df2['State Year Month'] = df2.index
df2

,pcpnst,pdsist,phdist,sp01st,sp02st,sp06st,sp12st,sp24st,tmaxst,tminst,tmpcst,zndxst,State Year Month
"(1, 1992, 1)",6.12,-0.28,0.97,0.62,0.46,-0.26,0.88,0.65,54.30,33.90,44.10,1.01,"(1, 1992, 1)"
"(1, 1992, 2)",5.99,-0.10,1.02,0.48,0.71,0.08,1.00,0.40,63.50,39.30,51.40,0.46,"(1, 1992, 2)"
"(1, 1992, 3)",4.03,-0.70,-0.70,-0.77,-0.31,-0.24,0.59,-0.06,67.80,42.00,54.90,-1.84,"(1, 1992, 3)"
"(1, 1992, 4)",2.91,-1.12,-1.12,-0.83,-1.18,-0.20,0.09,-0.11,74.50,48.50,61.50,-1.49,"(1, 1992, 4)"
"(1, 1992, 5)",1.95,-1.60,-1.60,-1.31,-1.57,-0.71,-0.96,-0.41,81.00,55.00,68.00,-1.77,"(1, 1992, 5)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(47, 2015, 8)",4.00,1.29,1.29,0.22,-0.02,0.02,-0.06,0.70,76.30,55.90,66.10,0.42,"(47, 2015, 8)"
"(47, 2015, 9)",4.39,1.23,1.23,0.52,0.50,0.58,0.12,1.07,75.20,54.10,64.70,0.21,"(47, 2015, 9)"
"(47, 2015, 10)",2.70,1.32,1.32,0.22,0.46,0.56,0.01,0.99,59.00,37.90,48.40,0.65,"(47, 2015, 10)"
"(47, 2015, 11)",3.42,1.93,1.93,1.37,1.09,0.68,0.30,1.12,48.20,30.50,39.40,2.25,"(47, 2015, 11)"


In [15]:
df2['fire count'] = df2.apply(lambda x: fire_summary_dict.get(x['State Year Month'],0),axis=1)

In [16]:
df2[['State', 'Year', 'Month']] = df2['State Year Month'].apply(pd.Series)
df2 = df2.drop('State Year Month', axis=1)

In [17]:
display(df2)

,pcpnst,pdsist,phdist,sp01st,sp02st,sp06st,sp12st,sp24st,tmaxst,tminst,tmpcst,zndxst,fire count,State,Year,Month
"(1, 1992, 1)",6.12,-0.28,0.97,0.62,0.46,-0.26,0.88,0.65,54.30,33.90,44.10,1.01,9.0,1,1992,1
"(1, 1992, 2)",5.99,-0.10,1.02,0.48,0.71,0.08,1.00,0.40,63.50,39.30,51.40,0.46,27.0,1,1992,2
"(1, 1992, 3)",4.03,-0.70,-0.70,-0.77,-0.31,-0.24,0.59,-0.06,67.80,42.00,54.90,-1.84,14.0,1,1992,3
"(1, 1992, 4)",2.91,-1.12,-1.12,-0.83,-1.18,-0.20,0.09,-0.11,74.50,48.50,61.50,-1.49,18.0,1,1992,4
"(1, 1992, 5)",1.95,-1.60,-1.60,-1.31,-1.57,-0.71,-0.96,-0.41,81.00,55.00,68.00,-1.77,10.0,1,1992,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(47, 2015, 8)",4.00,1.29,1.29,0.22,-0.02,0.02,-0.06,0.70,76.30,55.90,66.10,0.42,45.0,47,2015,8
"(47, 2015, 9)",4.39,1.23,1.23,0.52,0.50,0.58,0.12,1.07,75.20,54.10,64.70,0.21,20.0,47,2015,9
"(47, 2015, 10)",2.70,1.32,1.32,0.22,0.46,0.56,0.01,0.99,59.00,37.90,48.40,0.65,48.0,47,2015,10
"(47, 2015, 11)",3.42,1.93,1.93,1.37,1.09,0.68,0.30,1.12,48.20,30.50,39.40,2.25,24.0,47,2015,11


In [18]:
df2.to_csv('../Data/temp.csv',index=False)

In [19]:
df2.isnull().values.any()

False